In [5]:
from utilsSpacy import *


In [2]:
from torchtext.datasets import Multi30k
from functools import partial
from torchtext.datasets import multi30k

multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"
multi30k.URL["test"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/mmt16_task1_test.tar.gz"

multi30k.MD5["train"] = "20140d013d05dd9a72dfde46478663ba05737ce983f478f960c1123c6671be5e"
multi30k.MD5["valid"] = "a7aa20e9ebd5ba5adce7909498b94410996040857154dab029851af3a866da8c"
multi30k.MD5["test"] = "6d1ca1dba99e2c5dd54cae1226ff11c2551e6ce63527ebb072a1f70f72a5cd36"

In [3]:
RUN_EXAMPLES = True
def show_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        return fn(*args)

def execute_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        fn(*args)

# global variables used later in the script
spacy_de, spacy_en = show_example(load_tokenizers)
vocab_src, vocab_tgt = show_example(load_vocab, args=[spacy_de, spacy_en])

Finished.
Vocabulary sizes:
8316
6384


In [ ]:
def check_outputs(
    valid_dataloader,
    model,
    vocab_src,
    vocab_tgt,
    n_examples=15,
    pad_idx=2,
    eos_string="</s>",
):
    results = [()] * n_examples
#   resultsHggngFace = [()]*n_examples
    for idx in range(n_examples):
        print("\nExample %d ========\n" % idx)
        b = next(iter(valid_dataloader))
        rb = Batch(b[0], b[1], pad_idx)
        greedy_decode(model, rb.src, rb.src_mask, 64, 0)[0]

        src_tokens = [
            vocab_src.get_itos()[x] for x in rb.src[0] if x != pad_idx
        ]
        tgt_tokens = [
            vocab_tgt.get_itos()[x] for x in rb.tgt[0] if x != pad_idx
        ]

        print(
            "Source Text (Input)        : "
            + " ".join(src_tokens).replace("\n", "")
        )
        print(
            "Target Text (Ground Truth) : "
            + " ".join(tgt_tokens).replace("\n", "")
        )
        model_out = greedy_decode(model, rb.src, rb.src_mask, 72, 0)[0]
        model_txt = (
            " ".join(
                [vocab_tgt.get_itos()[x] for x in model_out if x != pad_idx]
            ).split(eos_string, 1)[0]
            + eos_string
        )
        print("Model Output               : " + model_txt.replace("\n", ""))
        results[idx] = (rb, src_tokens, tgt_tokens, model_out, model_txt)
        # Model Spacy
        #model_out = greedy_decode(modelSpacy, rb.src, rb.src_mask, 72, 0)[0]
        #model_txt = (
        #    " ".join(
        #        [vocab_tgt.get_itos()[x] for x in model_out if x != pad_idx]
        #    ).split(eos_string, 1)[0]
        #    + eos_string
        #)
        #print("Model Output               : " + model_txt.replace("\n", ""))
        #results[idx] = (rb, src_tokens, tgt_tokens, model_out, model_txt)
    return results#,resultsHggngFace


def run_model_example(n_examples=25):
    global vocab_src, vocab_tgt, spacy_de, spacy_en

    print("Preparing Data ...")
    _, valid_dataloader = create_dataloaders(
        torch.device("cpu"),
        vocab_src,
        vocab_tgt,
        spacy_de,
        spacy_en,
        batch_size=1,
        is_distributed=False,
    )

    print("Loading Trained Model ...")

    model = make_model(len(vocab_src), len(vocab_tgt), N=6)
    model.load_state_dict(
        torch.load("multi30k_model_final_Spacy.pt", map_location=torch.device("cpu"))
    )

    print("Checking Model Outputs:")
    example_data = check_outputs(
        valid_dataloader, model, vocab_src, vocab_tgt, n_examples=n_examples
    )
    return model, example_data


execute_example(run_model_example)

Preparing Data ...
Loading Trained Model ...
Checking Model Outputs:

Example 0 ========

Memory shape = torch.Size([1, 128, 512])
Source Text (Input)        : <s> Eine blonde Frau mit braunem Cowboyhut macht eine obszöne Geste in Richtung der Kamera . </s>
Target Text (Ground Truth) : <s> A blond women in a brown cowboy hat makes an obscene gesture toward the camera . </s>
Memory shape = torch.Size([1, 128, 512])
Model Output               : <s> A blond woman in a brown cowboy hat , doing an ollie , the back - up shirt . </s>

Example 1 ========

Memory shape = torch.Size([1, 128, 512])
Source Text (Input)        : <s> <unk> Gruppe von Erwachsenen bei einer Zusammenkunft , ein Mann mit einer Tätowierung wirft etwas . </s>
Target Text (Ground Truth) : <s> <unk> group of adults at a gathering , one man with tattoos is throwing something . </s>
Memory shape = torch.Size([1, 128, 512])
Model Output               : <s> Group of adults at a gathering at something , one man with a tattoo on 